### Survey

In [12]:
#%pip install 

  Using cached faiss_cpu-1.8.0-cp312-cp312-win_amd64.whl.metadata (3.8 kB)
Using cached faiss_cpu-1.8.0-cp312-cp312-win_amd64.whl (14.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [37]:
import requests
from dotenv import find_dotenv, load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

### Change questions to sound friendlier

In [36]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain.schema import (
    HumanMessage,
    SystemMessage,
    AIMessage,
)

In [39]:
repo_id = "HuggingFaceH4/zephyr-7b-beta"

load_dotenv(find_dotenv())

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.1,
)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\user\.cache\huggingface\token
Login successful


In [95]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant."
            )
        ),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)
question = "How important is hair health to you?"
messages = chat_template.format_messages(question=f"Rephrase this question to be informal: {question} Just return a question.")


In [96]:
res = llm.invoke(messages)
print(res)


Assistant: Hey, what's up? So, I'm just curious, how important is hair health to you? Like, do you go all out with hair masks and oils, or are you more of a wash-and-go kind of person? Let me know!


### testing question retrieval

#### idea: given an answer, the most similar question is returned. If no similar questions, idk is written. When no questions in database, return 'you have reached the end'. Problem: it doesnt work

In [32]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate

In [33]:
questions = ['Is the shampoo heavy?',
             'How would you rate the shampoo\'s scent?',
             'How is the shampoo\'s colour?']

In [34]:
ids = ['1','2','3']
vectorstore = FAISS.from_texts(
    questions, embedding=OpenAIEmbeddings(), ids = ids
)
retriever = vectorstore.as_retriever()

template = """Use only the following pieces of context to give a question at the end.
If you don't have the a question, just say that you don't know, don't try to make up a question. If no context, reply with 'You have reached the end of the survey'.

{context}

Question: {question}

Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [27]:
rag_chain.invoke("The shampoo's scent is good")

"How would you rate the shampoo's scent?"

In [28]:
vectorstore.delete('2')

True

In [29]:
# I realise that this is pointless and we can just retrieve the most 
# similar question based on cosine similarity since we are going to keep asking questions anyway
rag_chain.invoke("The shampoo's scent is good")

"I don't know."

In [30]:
# delete all questions in database
vectorstore.delete(['1','3'])

True

In [31]:
# making stuff up
rag_chain.invoke("The shampoo's scent is good")

'Do you like the scent of the shampoo?'

### Keep backlog of messages

In [ ]:
#nothing

### Converse with user

In [98]:
#nothing